<a href="https://colab.research.google.com/github/svandergoote/LGBIO2060-2021/blob/Tp2/LGBIO2060_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>